# TWITTER ANALYTICS 
## Text Classication & Sentiment Analysis
### Musarath J. Rahamathullah

# Table of contents
* [Libraries](#lib_header)
* [Data Collection](#Data-bullet)
* [Data Preprocessing](#DP-bullet)
* [Word Cloud](#wc-bullet)
* [Latent Dirichlet Allocation](#lda-bullet)
* [Text Classification Models](#cm-bullet)
    * [Navie Bayes](#nb-bullet)
    * [Linear Classifier](#lc-bullet)
    * [Support Vector Machine](#svm-bullet)
    * [Random Forest](#rf-bullet)
    * [Extreme Gradient Boosting](#xg-bullet)
    * [Convoluntional Neural Network](#cnn-bullet)
* [Classifier Interface](#ci-bullet)
    

## LIBRARIES <a class="anchor" id="lib_header"></a>

In [1]:
#load libraries
import tweepy
from tweepy import OAuthHandler
import json
import wget
import csv
import numpy as np
import pandas as pd
import re
import webbrowser

from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn import decomposition
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import xgboost, string

import tensorflow
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from datetime import datetime
from plotly import graph_objs as go
import plotly.tools as plotly_tools
from plotly.offline import init_notebook_mode,plot
from plotly import tools

import chart_studio 
chart_studio.tools.set_credentials_file(username='musarath', api_key='hN1B3D9TpmpUSxRG5iKy')
import chart_studio.plotly as py
import plotly.graph_objs as go

from flask import Flask, request, render_template, jsonify
import import_ipynb

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



Using TensorFlow backend.


## DATA COLLECTION<a class="anchor" id="Data-bullet"></a>

In [2]:
# get tweets from twitter 
def twitter_credentials(hashtag):
    # Twitter API credentials
    with open('twitter_credentials.json') as cred_data:
        info = json.load(cred_data)
        consumer_key = info['CONSUMER_KEY']
        consumer_secret = info['CONSUMER_SECRET']
        access_key = info['ACCESS_KEY']
        access_secret = info['ACCESS_SECRET']

    # Create the api endpoint
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth,wait_on_rate_limit=True)


    twts = []
    print("Downloading in progress...")
    for tweet in tweepy.Cursor( api.search, q='#' + hashtag  + "-filter:retweets",
                                #api.search, q='#' + hashtag + since:2019-09-01 + until:2019-9-22 +  "-filter:retweets" ,
                               lang="en").items(10000):
        twts.append([str(tweet.created_at),tweet.text.encode('utf-8')])
    print("Downloading completed!")

     
    return twts
 

In [3]:
# ### Uncomment these code to downlaod tweets ###
# # Mention the maximum number of tweets that you want to be extracted.
# #maximum_number_of_tweets_to_be_extracted = int(input('Enter the number of tweets that you want to extract- '))

# # Mention the hashtag that you want to look out for
# hashtag = input('Enter the hashtag you want to scrape- ')
# # tweets download
# twts_download = twitter_credentials(hashtag)

# print("Length of tweets:",len(twts_download))

In [4]:
# write to csv file
def write_tweets_to_file(hash,tweets):
    with open('tweets_with_hashtag_'+hash+'.csv', 'w',encoding='utf8',newline='') as the_file:
        writer = csv.writer(the_file)
        writer.writerow(['created_at', 'text'])
        writer.writerows(tweets)
        
# call the function        
#write_tweets_to_file(hashtag,twts_download)

In [5]:
# cross check the tweets saved
def read_tweets_from_file(hash):
    tweets = []
    with open('tweets_with_hashtag_'+hash+'.csv', 'r') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:
            if "fruit" not in row:
                tweets.append(row)
    
    return tweets

# # call read from file function
#tweets_data = read_tweets_from_file(hashtag)
#print ('Extracted ' + str(len(tweets_data)) + ' tweets with hashtag #' + hashtag)

In [6]:
# get_tweets = pd.DataFrame(tweets_data)
# get_tweets.rename(columns=get_tweets.iloc[0],inplace = True)
# get_tweets.drop(get_tweets.index[0],inplace = True)
# print(get_tweets['text'].head())


## DATA PREPROCESSING<a class="anchor" id="DP-bullet"></a>

In [7]:
# function to remove http link in tweet text
def remove_http(tweets):
    http_removed_tweets = []
    for idx,row in enumerate(tweets):
        http_removed_tweets.append(re.sub('[n]?http[s]?://\S+', '', row))
    return http_removed_tweets   


In [8]:
# Tweets text cleanup
#to remove the pattern ‘@user’ from all the tweets in our data.

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

def tweets_clean_up(get_tweets):

    get_tweets = pd.DataFrame(get_tweets)
    get_tweets.rename(columns=get_tweets.iloc[0],inplace = True)
    get_tweets.drop(get_tweets.index[0],inplace = True)
    #print(get_tweets.head())

    # remove http link
    get_tweets['tidy_tweet'] = remove_http(get_tweets['text'])

    # remove twitter handles (@user)
    get_tweets['tidy_tweet'] = np.vectorize(remove_pattern)(get_tweets['tidy_tweet'], "@[\w]*")
    
    # remove special characters, numbers, punctuations
    get_tweets['tidy_tweet'] = get_tweets['tidy_tweet'].str.replace("[^a-zA-Z]", " ")

        
    # removing the short words
    get_tweets['tidy_tweet'] = get_tweets['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
        
    return get_tweets
    
# # tweets cleanup function
#tweets_data = tweets_clean_up(tweets_data)

# # write to csv
# tweets_data.to_csv('tweets_with_cleaned_hashtag_'+hashtag+'.csv',index=False)
    

In [9]:
#tweets_data.head()

In [10]:
# tokenization method
def tweet_tokenization(tweet):
    # Tokenization
    tokenized_tweet = tweet['tidy_tweet'].apply(lambda x: x.split())    
    return tokenized_tweet

# # tokenize the tweets
#tweets_data['tokens'] =tweet_tokenization(tweets_data)
#tweets_data['tokens'].head()

In [11]:
#tweets_data.head()

In [12]:
# stemming / Lemmatizier
def tweet_reduce_wordforms(tokenized_tweet):
    #Stemming
    #tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
    #tokenized_tweet.head()
    
    lmtzr = WordNetLemmatizer()
    tokenized_tweet = tokenized_tweet.apply(lambda x: [lmtzr.lemmatize(i,'v') for i in x])
    
    for i in range(1,len(tokenized_tweet)+1):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    return tokenized_tweet

# # get tokens to baseform
#tweets_data['tokens'] = tweet_reduce_wordforms(tweets_data['tokens'])


In [13]:
#tweets_data['tokens']

## WORD CLOUD<a class="anchor" id="wc-bullet"></a>

In [14]:
# # word cloud method
# def tweets_word_cloud(tweets):
#     # worldcloud of tweets
#     all_words = ' '.join([text for text in tweets['tokens']])

#     #import worldCloud
#     wordcloud = WordCloud(width=800, height=500, random_state=21,background_color= 'white',max_words=100, max_font_size=110).generate(all_words)

#     plt.figure(figsize=(10, 7))
#     plt.imshow(wordcloud, interpolation="bilinear")
#     plt.axis('off')
#     plt.show()tweets_daily_count = df_tweets.groupby('created_at')['tidy_tweet'].count()
# tweets_daily_count.plot(kind='bar', x='Date')
    
# #tweets_word_cloud(tweets_data)

In [15]:
# tmp = tweets_data

# tmp['Date'] = tmp.created_at.apply(lambda x: pd.Series(str(x).split(" ")[0]))
# #print(d)
# tweets_daily_count = tmp.groupby('Date')['tidy_tweet'].count()
# pt = tweets_daily_count.plot(kind='bar', x='Date')
# # Set the y-axis label
# pt.set_ylabel("Number of tweets")

## LATENT DIRICHLET ALLOCATION<a class="anchor" id="lda-bullet"></a>

In [16]:
# #read all the titles
# titles = []
# for tweet in tweets_data['tokens']:
 
#     titles.append(tweet) 
    
# print(titles)

In [17]:
# # create a count vectorizer object 
# count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
# count_vect.fit(titles)

# # transform the training and validation data using count vectorizer object
# xtrain_count =  count_vect.transform(titles)


In [18]:
# # train a LDA Model
# def lda_mode(title):

#     lda_model = decomposition.LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=20)
#     X_topics = lda_model.fit_transform(xtrain_count)
#     topic_word = lda_model.components_ 
#     vocab = count_vect.get_feature_names()

#     # view the topic models
#     n_top_words = 10
#     topic_summaries = []
#     for i, topic_dist in enumerate(topic_word):
#         topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#         topic_summaries.append(' '.join(topic_words))
        
#     return topic_summaries

# topic_summary = lda_mode(titles)
# topic_summary

## TEXT CLASSIFICATION MODELS<a class="anchor" id="cm-bullet"></a>

In [19]:
model_train_data = pd.read_csv("tweets_with_cleaned_hashtag_samsung_classification.csv")
train_DF = pd.DataFrame()
train_DF['tweet_text'] = model_train_data['tidy_tweet']
train_DF['label'] = model_train_data['Abstract']

In [20]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_DF['tweet_text'], train_DF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [21]:
# word level tf-idf
tfidf_word = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_word.fit(train_DF['tweet_text'])

xtrain_tfidf_word =  tfidf_word.transform(train_x)
xvalid_tfidf_word =  tfidf_word.transform(valid_x)

# ngram level tf-idf 
tfidf_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_ngram.fit(train_DF['tweet_text'])
xtrain_tfidf_ngram =  tfidf_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_ngram.transform(valid_x)

# characters level tf-idf
tfidf_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_chars.fit(train_DF['tweet_text'])
xtrain_tfidf_chars =  tfidf_chars.transform(train_x) 
xvalid_tfidf_chars =  tfidf_chars.transform(valid_x)

In [22]:
# train_DF['char_count'] = train_DF['tweet_text'].apply(len)
# train_DF['word_count'] = train_DF['tweet_text'].apply(lambda x: len(x.split()))
# train_DF['word_density'] = train_DF['char_count'] / (train_DF['word_count']+1)
# train_DF['punctuation_count'] = train_DF['tweet_text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
# train_DF['title_word_count'] = train_DF['tweet_text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
# train_DF['upper_case_word_count'] = train_DF['tweet_text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [23]:
#train_DF.head()

In [24]:
def train_classifier_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.classification_report(predictions, valid_y)
#accuracy_score(predictions, valid_y)

### NAIVE BAYES<a class="anchor" id="nb-bullet"></a>

In [25]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_classifier_model(naive_bayes.MultinomialNB(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
print ("Naive Bayes, WordLevel TF-IDF:\n" , accuracy)
#print ("Naive Bayes, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Naive Bayes, N-Gram Vectors:\n" , accuracy)
#print( "Navie Bayes, N-Gram Vectors: {}%".format(round(accuracy*100,2)))

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_classifier_model(naive_bayes.MultinomialNB(), xtrain_tfidf_chars, train_y, xvalid_tfidf_chars)
print ("Naive Bayes, CharLevel TF-IDF:\n" , accuracy)
#print ("Navie Bayes, CharLevel Vectors: {}%".format(round(accuracy*100,2)))

Naive Bayes, WordLevel TF-IDF:
               precision    recall  f1-score   support

           0       0.61      0.96      0.75        28
           1       0.42      0.64      0.50        44
           2       0.96      0.47      0.63       154
           3       0.29      1.00      0.45        13
           4       0.00      0.00      0.00         0

    accuracy                           0.59       239
   macro avg       0.46      0.61      0.47       239
weighted avg       0.78      0.59      0.61       239

Naive Bayes, N-Gram Vectors:
               precision    recall  f1-score   support

           0       0.45      1.00      0.62        20
           1       0.36      0.63      0.46        38
           2       0.88      0.39      0.54       172
           3       0.20      1.00      0.33         9
           4       0.00      0.00      0.00         0

    accuracy                           0.50       239
   macro avg       0.38      0.60      0.39       239
weighted avg   

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



### LINEAR CLASSIFIER<a class="anchor" id="lc-bullet"></a>

In [26]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_classifier_model(linear_model.LogisticRegression(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
#print( "Logistic Regression, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))
print( "Logistic Regression, WordLevel TF-IDF:\n",accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print ("Logistic Regression, N-Gram Vectors: {}%".format(round(accuracy*100,2)))
print ("Logistic Regression, N-Gram Vectors: \n",accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_classifier_model(linear_model.LogisticRegression(), xtrain_tfidf_chars, train_y, xvalid_tfidf_chars)
#print( "Logistic Regression, CharLevel Vectors:{}%".format(round(accuracy*100,2)))
print( "Logistic Regression, CharLevel Vectors:\n",accuracy)

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



Logistic Regression, WordLevel TF-IDF:
               precision    recall  f1-score   support

           0       0.57      0.93      0.70        27
           1       0.73      0.67      0.70        73
           2       0.95      0.63      0.76       114
           3       0.51      0.92      0.66        25
           4       0.00      0.00      0.00         0

    accuracy                           0.71       239
   macro avg       0.55      0.63      0.56       239
weighted avg       0.79      0.71      0.72       239

Logistic Regression, N-Gram Vectors: 
               precision    recall  f1-score   support

           0       0.34      1.00      0.51        15
           1       0.33      0.76      0.46        29
           2       0.92      0.37      0.53       187
           3       0.18      1.00      0.30         8
           4       0.00      0.00      0.00         0

    accuracy                           0.48       239
   macro avg       0.35      0.63      0.36       23

### SUPPORT VECTOR MACHINE<a class="anchor" id="svm-bullet"></a>

In [27]:
# SVM on Word Level TF IDF Vectors
accuracy = train_classifier_model(svm.SVC(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
#print( "SVM, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))
print( "SVM, WordLevel TF-IDF:/n%",accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print ("SVM, N-Gram Vectors: {}%".format(round(accuracy*100,2)))
print ("SVM, N-Gram Vectors:\n%",accuracy)


# SVM on Character Level TF IDF Vectors
accuracy = train_classifier_model(svm.SVC(), xtrain_tfidf_chars, train_y, xvalid_tfidf_chars)
#print( "SVM, CharLevel Vectors: {}%".format(round(accuracy*100,2)))
print( "SVM, CharLevel Vectors: \n",accuracy)

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\sv

SVM, WordLevel TF-IDF:/n%               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.32      0.48       239
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.32       239
   macro avg       0.20      0.06      0.10       239
weighted avg       1.00      0.32      0.48       239

SVM, N-Gram Vectors:
%               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.32      0.48       239
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.32       239
   macro avg       0.20      0.06      0.10       239
weighted avg       1.00     

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



### RANDOM FOREST<a class="anchor" id="rf-bullet"></a>

In [28]:
# RF on Word Level TF IDF Vectors
accuracy = train_classifier_model(ensemble.RandomForestClassifier(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
#print ("Random Forest, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))
print ("Random Forest, WordLevel TF-IDF: \n%",accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print ("Random Forest, N-Gram Vectors: {}%".format(round(accuracy*100,2)))
print ("Random Forest, N-Gram Vectors: \n",accuracy)


# SVM on Character Level TF IDF Vectors
accuracy = train_classifier_model(ensemble.RandomForestClassifier(), xtrain_tfidf_chars,train_y,xvalid_tfidf_chars)
#print( "Random Forest, CharLevel Vectors: {}%".format(round(accuracy*100,2)))
print( "Random Forest, CharLevel Vectors: \n",accuracy)

Random Forest, WordLevel TF-IDF: 
%               precision    recall  f1-score   support

           0       0.73      0.84      0.78        38
           1       0.91      0.62      0.74        98
           2       0.80      0.87      0.84        70
           3       0.58      0.87      0.69        30
           4       0.29      0.67      0.40         3

    accuracy                           0.76       239
   macro avg       0.66      0.77      0.69       239
weighted avg       0.80      0.76      0.76       239

Random Forest, N-Gram Vectors: 
               precision    recall  f1-score   support

           0       0.43      1.00      0.60        19
           1       0.81      0.36      0.50       148
           2       0.51      0.65      0.57        60
           3       0.22      0.83      0.35        12
           4       0.00      0.00      0.00         0

    accuracy                           0.51       239
   macro avg       0.39      0.57      0.41       239
weighted

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



### EXTREME GRADIENT BOOSTING<a class="anchor" id="xg-bullet"></a>

In [29]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_classifier_model(xgboost.XGBClassifier(), xtrain_tfidf_word.tocsc(), train_y, xvalid_tfidf_word.tocsc())
#print( "Extereme Gradient Boosting, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))
print( "Extereme Gradient Boosting, WordLevel TF-IDF: \n",accuracy)

# Extereme Gradient Boosting on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram.tocsc(), train_y, xvalid_tfidf_ngram.tocsc())
#print( "Extereme Gradient Boosting, N-Gram Vectors: {}%".format(round(accuracy*100,2)))
print( "Extereme Gradient Boosting, N-Gram Vectors: \n",accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_classifier_model(xgboost.XGBClassifier(), xtrain_tfidf_chars.tocsc(), train_y, xvalid_tfidf_chars.tocsc())
#print( "Extereme Gradient Boosting, CharLevel Vectors: {}%".format(round(accuracy*100,2)))
print( "Extereme Gradient Boosting, CharLevel Vectors: \n",accuracy)

Extereme Gradient Boosting, WordLevel TF-IDF: 
               precision    recall  f1-score   support

           0       0.70      0.84      0.77        37
           1       0.97      0.65      0.78       100
           2       0.79      1.00      0.88        60
           3       0.80      0.92      0.86        39
           4       0.29      0.67      0.40         3

    accuracy                           0.81       239
   macro avg       0.71      0.82      0.74       239
weighted avg       0.85      0.81      0.81       239

Extereme Gradient Boosting, N-Gram Vectors: 
               precision    recall  f1-score   support

           0       0.43      1.00      0.60        19
           1       0.36      0.71      0.48        34
           2       0.89      0.40      0.56       168
           3       0.24      0.69      0.36        16
           4       0.00      0.00      0.00         2

    accuracy                           0.51       239
   macro avg       0.39      0.56    

In [30]:
def train_classifier_final_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
        
    return predictions


In [31]:
def read_new_tweets(file):
    tweets = []
    with open(file, 'r') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:
            tweets.append(row)
    
    return tweets


## CLASSIFIER INTERFACE<a class="anchor" id="ci-bullet"></a>

In [32]:
#import HTML
import flask_ngrok
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run

# to get information from my-form page
@app.route('/')
def my_form():
    return render_template('my-form.html')

# to post the result to web page
@app.route('/',methods=['POST'])
def my_analysis():
   
    # read the batch file
    file = request.form['fileupload']    # get the url and store in varaible
    print(file)
    # read tweets from file
    tweets = read_new_tweets(file)
    # clean up the tweets
    tweets = tweets_clean_up(tweets)
    # tokenize
    tokens_tfidf =  tfidf_word.transform(tweets['tidy_tweet'])
    # predict the tags using classifier model    
    pred = train_classifier_final_model(xgboost.XGBClassifier(), xtrain_tfidf_word.tocsc(), train_y, tokens_tfidf.tocsc())
    pred = encoder.inverse_transform(pred)
    # convert to dataframe
    output = pd.DataFrame({"Tweet":tweets['tidy_tweet'],"Tag":pred})
    #print(output)
       
    
    return render_template('my-form.html',result = output.to_html())

# main function
if __name__ == '__main__':
    app.run(host='localhost',port=5051)
    #app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5051/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Nov/2019 17:18:00] "GET / HTTP/1.1" 200 -


C:\Users\Musarath\Musarath\Anly699_Project\example.csv


127.0.0.1 - - [30/Nov/2019 17:18:41] "POST / HTTP/1.1" 200 -


## Twitter APPLICATION

### The below code is for Twitter Analytics

In [33]:
# twitter_tweets = []
# # download data
# def twitter_data_download(hash_tag):    
    
#     # tweets download
#     twts_download = twitter_credentials(hash_tag)

#     # call the function        
#     write_tweets_to_file(hash_tag,twts_download)

#     # call read from file function
#     twitter_tweets = read_tweets_from_file(hash_tag)
#     #print(twitter_tweets)
    
#     # tweets cleanup function
#     twitter_tweets = tweets_clean_up(twitter_tweets)
#     #print(twitter_tweets)
    
#     return twitter_tweets

# # call twitter data downlaod
# #hashtag = "samsung"
# #df_tweets = twitter_data_download(hashtag)


In [34]:
# df_tweets_data = []
# # function to tag tweets
# def twitter_abstract(tweets_data,xtrain_tfidf_word,train_y):
#     #tokenize
#     tokens_tfidf_word =  tfidf_word.transform(tweets_data['tidy_tweet'])

#     pred = train_classifier_final_model(xgboost.XGBClassifier(), xtrain_tfidf_word.tocsc(), train_y, tokens_tfidf_word.tocsc())
#     tweets_data['Abstract_class'] = pred
#     tweets_data['Abstract'] = encoder.inverse_transform(pred)
    
#     #convert the date field to date format 
#     df_tweets_data = pd.DataFrame(tweets_data) 

#     #convert to date type
#     df_tweets_data['created_at'] = pd.to_datetime(df_tweets_data["created_at"])
#     df_tweets_data['created_at'] = df_tweets_data['created_at'].dt.date 
#     # drop the columns
#     df_tweets_data.drop(['text','Abstract_class'],axis = 1,inplace = True)
#     return df_tweets_data

# # call twitter classifier
# #df_tweets = twitter_abstract(df_tweets,xtrain_tfidf_word,train_y)

In [35]:
# # method to calculate sentiment of each tweet
# def tweets_sentiment(df_tweet):
#     # next, we initialize VADER so we can use it within our Python script
#     sid = SentimentIntensityAnalyzer()
#     sentiment_score = []

#     ## Calling the polarity_scores method on sid and passing in the title 
#     #outputs a dictionary with negative, neutral, positive, and compound scores for the input title
#     for tweet in df_tweet['tidy_tweet']:
#         scores = sid.polarity_scores(tweet)
#         sentiment_score.append(scores['compound'])

#     df_tweet['sentiment'] = sentiment_score
#     return df_tweet

# # sentiment
# #df_tweets = tweets_sentiment(df_tweets)

In [36]:
# #c onvert datatime to date
# df_tweets['Date'] = df_tweets.created_at.apply(lambda x: pd.Series(str(x).split(" ")[0]))

# df_average_sentiment = df_tweets.groupby('Date')['sentiment'].mean()
# pt = df_average_sentiment.plot(kind='bar', x='Date',colormap = 'Paired')
# # Set the y-axis label
# pt.set_ylabel("Sentiment_score")

In [37]:
# tweets_data = tweets_polarity(tweets_data)
# tweets_data.index.name = 'Id'
# #tweets_data.reset_index(drop=True, inplace=True)
# tweets_data.head()

In [38]:
# # function to define polarity
# def tweets_polarity(df_tweets):
#     polarity = []
#     for t in df_tweets['sentiment']:
#         if t<0:
#             polarity.append("Negative")
#         elif t == 0:
#             polarity.append("Neutral")
#         else:
#             polarity.append("Positive")
#     df_tweets['Polarity'] = polarity
#     return df_tweets

# # call polarity
# #df_tweets = tweets_polarity(df_tweets)

In [39]:
# unique_elements, counts_elements = np.unique(df_tweets['Abstract'], return_counts=True)
# counts_elements

In [40]:
# # function for number of tweets per abstract over time
# def tweets_over_time(df_tweets):
#     df_daily_count = df_tweets.groupby(['created_at','Abstract'],as_index = False).agg({'tidy_tweet': "count"})

#     tmp = df_daily_count[df_daily_count['Abstract'] == 'CustomerService']
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(
#         x=tmp['created_at'], y=tmp['tidy_tweet'],
#         hoverinfo='x+y',
#         mode='lines',
#         line=dict(width=0.5,
#                   color='rgb(131, 90, 241)'),
#         name = 'CustomerService',
#         stackgroup='one' # define stack group
#     ))
#     tmp1 = df_daily_count[df_daily_count['Abstract'] == 'News']
#     fig.add_trace(go.Scatter(
#         x=tmp1['created_at'], y=tmp1['tidy_tweet'],
#         hoverinfo='x+y',
#         mode='lines',
#         line=dict(width=0.5,
#             color='rgb(127, 166, 238)'), 
#         name = 'News',
#         stackgroup='one'
#     ))
#     tmp2 = df_daily_count[df_daily_count['Abstract'] == 'Price']
#     fig.add_trace(go.Scatter(
#         x=tmp2['created_at'], y=tmp2['tidy_tweet'],
#         hoverinfo='x+y',
#         mode='lines',
#         line=dict(width=0.5,
#             color='rgb(111, 231, 219)'), 
#         name = 'Price',
#         stackgroup='one'
#      ))
#     tmp3 = df_daily_count[df_daily_count['Abstract'] == 'Products']
#     fig.add_trace(go.Scatter(
#         x=tmp3['created_at'], y=tmp3['tidy_tweet'],
#         hoverinfo='x+y',
#         mode='lines',
#         line=dict(width=0.5, 
#             color='rgb(184, 247, 212)'),
#         name = 'Products',
#         stackgroup='one'
#      ))
#     tmp4 = df_daily_count[df_daily_count['Abstract'] == 'Recommendation']
#     fig.add_trace(go.Scatter(
#         x=tmp4['created_at'], y=tmp4['tidy_tweet'],
#         hoverinfo='x+y',
#         mode='lines',
#         line=dict(width=0.5, 
#             color='rgb(95,158,160)'),
#         name = 'Recommendation',
#         stackgroup='one'
#      ))
#     fig.layout.update(title_text='Total activity over time',
#                       paper_bgcolor='rgba(0,0,0,0)',
#                     plot_bgcolor='rgba(0,0,0,0)',
#                      margin=dict(l=50, r=10, t=80, b=80))
#     fig.update_layout(legend_orientation="h")

    
#     plot_url = py.plot(fig, filename='Tweets over time', auto_open=False,)
#     #first_plot_url = fig.write_html('first_figure.html', auto_open=True)
#     #print(plot_url)
#     #fig.show()
#     return plot_url
#     #return fig


In [41]:
# call twitter activity plot
# tweets_over_time_plot_url = tweets_over_time(df_tweets)
# tweets_over_time_plot_url

In [42]:
# # method to plot sentiments over all categories
# def sentiment_analysis_plot(df_tweets,hashtag):
#     #plot sentiments w.r.t abstract
#     number_of_tweets = str(len(df_tweets['tidy_tweet']))
#     df_sentiment_polarity = df_tweets.groupby(['Abstract','Polarity'],as_index = False).agg({'tidy_tweet': "count"})

#     tmp = df_sentiment_polarity[df_sentiment_polarity['Polarity'] == 'Positive']
#     fig = go.Figure()
#     fig.add_trace(go.Bar(    
#         y=tmp['Abstract'],
#         x=tmp['tidy_tweet'] ,
#         name='Positive',
#         orientation='h',
#         marker=dict(
#             color='green',
#         )
#     ))
#     tmp1 = df_sentiment_polarity[df_sentiment_polarity['Polarity'] == 'Neutral']
#     fig.add_trace(go.Bar(    
#         y=tmp1['Abstract'],
#         x=tmp1['tidy_tweet'],
#         name='Neutral',
#         orientation='h',
#         marker=dict(
#             color='gold',
#         )
#     ))
#     tmp2 = df_sentiment_polarity[df_sentiment_polarity['Polarity'] == 'Negative']
#     fig.add_trace(go.Bar(
#         y=tmp2['Abstract'],
#         x=tmp2['tidy_tweet'],
#         name='Negative',
#         orientation='h',
#         marker=dict(
#             color='red',
#         )
#     ))
#     fig.layout.update(barmode='stack',
#         paper_bgcolor='rgba(0,0,0,0)',
#         plot_bgcolor='rgba(0,0,0,0)',
#         #paper_bgcolor='rgb(248, 248, 255)',
#         #plot_bgcolor='rgb(248, 248, 255)',
#         margin=dict(l=100, r=10, t=80, b=80),
#     )
#     fig.layout.update(title_text='Data gathered from the analysis of '+ number_of_tweets +' twitter reviews on '+ hashtag.upper())
#     fig.update_layout(legend_orientation="h")

#     #fig.show()
#     plot_url = py.plot(fig, filename='Tweets Sentiment', auto_open=False,)
    
#     return plot_url
#     #return fig

In [43]:
# call sentiment plot
#sentiment_analysis_plot(df_tweets,hashtag)

In [44]:
# def sentiment_categorize(df_tweets):
#     # categorize sentiment into different categories
#     tweets_sent_cat = []
#     #df_tweets_1 = {'sentiment': [0.35,0.4]}
#     #df_tweets_1 = pd.DataFrame(df_tweets_1)
#     for t in df_tweets['sentiment']:
#         if t == 0:
#             tweets_sent_cat.append("fair") 
#         elif t <= -0.9:
#             tweets_sent_cat.append("Unpleasant")
#         elif t > -0.9 and t <= -0.7:
#              tweets_sent_cat.append("disgust")
#         elif t > -0.7 and t <= -0.4:
#              tweets_sent_cat.append("upset")
#         elif t < 0 and t > -0.4:
#              tweets_sent_cat.append("sadness")
#         elif t >= 0.9:
#             tweets_sent_cat.append("pleasant")
#         elif t >= 0.7 and t < 0.9:
#              tweets_sent_cat.append("elate")
#         elif t >= 0.4 and t < 0.7:
#              tweets_sent_cat.append("excited")
#         elif t >0 and t < 0.4:
#              tweets_sent_cat.append("happy")

#     df_tweets['tweets_sent_cat'] =  tweets_sent_cat
    
#     return df_tweets

# # call sentiment  categories
# #df_tweets = sentiment_categorize(df_tweets)

In [45]:
# import plotly.express as px
# # method to plot spiral plot of sentiments
# def tweets_spiral(df_tweets):
#     df_tweets_sent_cat = df_tweets.groupby(['Abstract','tweets_sent_cat'],as_index = False).agg({'tidy_tweet': "count"})
#     #df_tweets_sent_cat

#     #plot the chart
#     fig = px.bar_polar(df_tweets_sent_cat, r="tidy_tweet", theta="tweets_sent_cat",
#                         color="Abstract",template="seaborn",
#                        color_discrete_sequence= px.colors.sequential.Plasma[-2::-1])
#     fig.layout.update(title_text='Sentiments spiral')
#     fig.update_layout(legend_orientation="h",
#                       margin=dict(l=80, r=80, t=80, b=100),
#                      paper_bgcolor='rgba(0,0,0,0)')
    
#     #fig.show()
#     plot_url = py.plot(fig, filename='Tweets Spiral Sentiment', auto_open=False,)
    
#     return plot_url

In [46]:
# # call sentiment spliral plot
# tweets_spiral(df_tweets)

In [47]:
# # method to display all tweets for the given hashtag in a table
# def tweets_table(df_tweets,hashtag):
#     df_tweets = df_tweets.loc[1:3000]
#     values = [list(df_tweets['created_at']), #1st col
#       list(df_tweets['tidy_tweet']),
#       list(df_tweets['Abstract'])]


#     fig = go.Figure(data=[go.Table(
#       columnorder = [1,2,3],
#       columnwidth = [80,400,80],
#       header = dict(
#         values = [['<b>CREATED_AT</b>'],
#                     ['<b>TWEETS</b>'],
#                  ['<b>ABSTRACT</b>']],
#         line_color='darkslategray',
#         fill_color='royalblue',
#         align=['left','center'],
#         font=dict(color='white', size=12),
#         height=40
#       ),
#       cells=dict(
#         values=values,
#         line_color='darkslategray',
#         fill=dict(color=['white','white', 'paleturquoise']),
#         align=['left', 'center'],
#         font_size=12,
#         height=30)
#         )
#     ])
#     fig.layout.update(title_text='Tweets on #'+hashtag.upper(),
#                       paper_bgcolor='rgba(0,0,0,0)',
#                      font=dict(
#                              family="Courier New, monospace",
#                              size=18,
#                              color="#7f7f7f"
#             ))

    
#     #fig.show()
#     plot_url = py.plot(fig, filename='Tweets Table', auto_open=False,weight = "bold")
    
#     return plot_url
#     #return fig

In [48]:
# get tweets table
#tweets_table(df_tweets,hashtag)

In [49]:
# def define_html_string(hashtag,p1,p2,t1,p3):
#     print(hashtag)
#     html_string = '''
#     <html>
#         <head>
#             <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
#             <style>body{ margin:0 100; background:whitesmoke; }</style>
#         </head>
#         <body>
#             <h1><img src = "https://1000logos.net/wp-content/uploads/2017/06/Twitter-Logo.png" alt = "Twitter" 
#             width="100" height="100">Twitter Review Analysis on #'''+ hashtag.upper() + ''' </h1> 
#             <!-- Add icon library -->
        
            
#             <!-- *** Section 1 *** ---> 
#             <!--<h2>Tweets over time</h2>-->
#             <iframe width="700" height="400" frameborder="0" seamless="seamless" scrolling="no" \
#     src="''' + p1 + '''"></iframe>
#              <!-- *** Section 2 *** --->
#             <!--<h2>Tweets Sentiment Analysis</h2>-->
#             <iframe width= "600" height="400" frameborder="0" seamless="seamless" scrolling="no" \
#     src="''' + p2 + '''"></iframe> 
#             <!--<h2>Twitter Reviews</h2>-->
#             <iframe width="800" height="500" frameborder="0" seamless="seamless" scrolling="no" \
#     src="''' + t1 + '''"></iframe> 
#             <!--<h2>Sentiment Spiral</h2>-->
#             <iframe width="500" height="500" frameborder="0" seamless="seamless" scrolling="no" \
#     src="''' + p3 + '''"></iframe> 

#         </body>
#     </html>'''
    
#     return html_string

In [50]:
# # method to create a report html
# def write_to_html(hashtag,html_string):
#     f = open('C:/Users/Musarath/Musarath/Anly699_Project/Templates/'+hashtag+'_report.html','w')
#     f.write(html_string)
#     f.close()
    
#     return

In [51]:
## unblock this code to test locally ##

# import chart_studio
# import plotly.express as px
# #chart_studio.tools.set_credentials_file(username='musarath', api_key='V2wKW2XdT3cggRuR3916')
# #import plotly.plotly as py
# hashtag = "mus"
# # plot generations
# df_tweets = sentiment_categorize(df_tweets)

# tweets_over_time_plot_url = tweets_over_time(df_tweets)
# #print(tweets_over_time_plot_url)
# tweets_sentiment_plot_url = sentiment_analysis_plot(df_tweets,hashtag)
# #print(tweets_sentiment_plot_url)
# all_tweets_table = tweets_table(df_tweets,hashtag)
# #print(all_tweets_table)
# sentiment_spiral_plot_url = tweets_spiral(df_tweets)
# #print(sentiment_spiral_plot_url)
# html_string = define_html_string(tweets_over_time_plot_url,tweets_sentiment_plot_url,all_tweets_table,sentiment_spiral_plot_url)
# write_to_html(hashtag,html_string)

In [52]:
# total number of positive, negative and neutral tweets
#np.unique(df_tweets['Polarity'], return_counts=True)

In [53]:
# Abstract sentiment
# df_sentiments = pd.DataFrame(df_tweets.groupby('Abstract')['sentiment'].mean(),columns = ['sentiment'])
# df_sentiments.reset_index() 
# df_sentiments['sentiment']

In [54]:
# #plot sentiments w.r.t abstract
# number_of_tweets = str(len(df_tweets['tidy_tweet']))
# df_sentiment_polarity = df_tweets.groupby(['Abstract','Polarity'],as_index = False).agg({'tidy_tweet': "count"})

# fig = px.bar(df_sentiment_polarity, x="Abstract", y="tidy_tweet", color=["green","gold","red"])
# fig.update_layout(title_text='Data gathered from the analysis of '+ number_of_tweets +' twitter reviews on '+ hashtag)
# fig.show()

In [55]:
# # tweets scatter plot
# fig = go.Figure(data=go.Scatter(
#     y = df_tweets['sentiment'],
#     mode='markers',
#     marker=dict(
#         size=16,
#         color= df_tweets['sentiment'], #set color equal to a variable
#         colorscale='Viridis', # one of plotly colorscales
#         showscale=True
#     )
# ))

# fig.show()

In [56]:
# global str

# app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run

# # to get information from my-form page
# @app.route('/')
# def my_form():
#     return render_template('my-report.html')

# # to post the result to web page
# @app.route('/',methods=['POST'])
# def my_hashtag_report():
#     # read the batch file
#     hashtag = str(request.form['string'])   # get the url and store in varaible
#     print(hashtag)

#     # get data
#     tweets_data = twitter_data_download(hashtag)
#     # tag the tweets
#     df_tweets = twitter_abstract(tweets_data,xtrain_tfidf_word,train_y)

#     # sentiment
#     df_tweets = tweets_sentiment(df_tweets)
#     # polarity
#     df_tweets = tweets_polarity(df_tweets)
#     # sentiment categorize
#     df_tweets = sentiment_categorize(df_tweets)

#     # plot generations
#     tweets_over_time_plot_url = tweets_over_time(df_tweets)
#     tweets_sentiment_plot_url = sentiment_analysis_plot(df_tweets,hashtag)
#     all_tweets_table = tweets_table(df_tweets,hashtag)
#     sentiment_spiral_plot_url = tweets_spiral(df_tweets)

#     # create report
#     html_string = define_html_string(hashtag,tweets_over_time_plot_url,tweets_sentiment_plot_url,all_tweets_table,sentiment_spiral_plot_url)
#     write_to_html(hashtag,html_string)

#     # open the report
#     #webbrowser.open_new_tab( 'C:/Users/Musarath/Musarath/Anly699_Project/'+hashtag+'_report.html')    

#     filename = str(hashtag+'_report.html')
#     return render_template(filename)
        
        
# # main function
# if __name__ == '__main__':
#     #app.run(host='localhost',port=5051)
#     app.run()

In [57]:
# # positive word cloud
# normal_words =' '.join([text for text in df_tweets['tidy_tweet'][df_tweets['sentiment'] > 0]])

# wordcloud = WordCloud(width=800, height=500, random_state=123, max_font_size=110,background_color= 'white',max_words=50).generate(normal_words)
# plt.figure(figsize=(10, 7))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis('off')
# plt.show()

In [58]:
# # negative word cloud
# negative_words = ' '.join([text for text in df_tweets['tidy_tweet'][df_tweets['sentiment'] < 0]])
# wordcloud = WordCloud(width=800, height=500,
# random_state=21, max_font_size=110,background_color= 'white',max_words=50).generate(negative_words)
# plt.figure(figsize=(10, 7))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis('off')
# plt.show()